In [1]:
import numpy as np

In [2]:
with open('xids.npy', 'rb') as f:
    Xids = np.load(f)
with open('xmask.npy', 'rb') as f:
    Xmask = np.load(f)
    Xmask = Xmask.astype(np.in32)
with open('labels.npy', 'rb') as f:
    labels = np.load(f)

In [3]:
import tensorflow as tf

In [4]:
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))

In [5]:
def map_function(ids, masks, labels):
    return {"input_ids": ids, "attention_mask": masks}, labels

dataset = dataset.map(map_function)

In [6]:
dataset = dataset.shuffle(100000).batch(32)

In [7]:
DS_LEN = len(dataset)

In [8]:
SPLIT = 0.85

train = dataset.take(round(DS_LEN*SPLIT))
val = dataset.skip(round(DS_LEN*SPLIT))

del dataset

In [9]:
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf

bert = TFAutoModel.from_pretrained("bert-base-cased")

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [22]:
SEQ_LEN = 50

input_ids = tf.keras.layers.Input(shape = (SEQ_LEN,), name = "Input IDS Layer", dtype= np.int32)
attention_mask = tf.keras.layers.Input(shape = (SEQ_LEN,), name = "Attention Mask Layer", dtype= np.int32)
actual_bert = bert(input_ids, attention_mask)